## Analysis of spam SMS messages (data from [here](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/))

In [1]:
import pandas as pd
import sklearn

%matplotlib inline

In [2]:
df=pd.read_csv('SMSSpamCollection',sep='\t',header=None,names=['class','text'])

In [3]:
df.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Split into test data and training data

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['class'], test_size=0.25)

## Some Pre-processing

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer

In [7]:
count_vect = CountVectorizer()

In [8]:
X_train_counts = count_vect.fit_transform(X_train)

In [9]:
list(count_vect.vocabulary_.items())[0:3]

[('desires', 2194), ('going', 3074), ('to', 6707)]

In [10]:
len(count_vect.vocabulary_)

7488

In [11]:
lab_bin=LabelBinarizer()
y_train_bin=lab_bin.fit_transform(y_train)
y_test_bin=lab_bin.fit_transform(y_test)

## Train

In [12]:
from sklearn.naive_bayes import MultinomialNB

In [13]:
clf = MultinomialNB().fit(X_train_counts, y_train_bin)

c:\program files\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
len(clf.coef_[0])

7488

In [15]:
import collections

In [16]:
importanceCount=collections.Counter()

In [17]:
for word,imp in zip(count_vect.vocabulary_.keys(),clf.coef_[0]):
    importanceCount[word]=imp

In [18]:
importanceCount.most_common()[-10:]

[('hubby', -9.928375268043908),
 ('fiting', -9.928375268043908),
 ('load', -9.928375268043908),
 ('mj', -9.928375268043908),
 ('lodge', -9.928375268043908),
 ('laden', -9.928375268043908),
 ('wrecked', -9.928375268043908),
 ('jacuzzi', -9.928375268043908),
 ('rip', -9.928375268043908),
 ('uterus', -9.928375268043908)]

## Now test

In [19]:
X_test_counts = count_vect.transform(X_test)

In [20]:
pred=clf.predict(X_test_counts)

In [21]:
from sklearn.metrics import average_precision_score

In [22]:
average_precision_score(y_test_bin,pred)

0.8922097571488308

## Sanity check

In [23]:
clf.predict(count_vect.transform(['win big on this offer']))

array([1])

In [24]:
clf.predict(count_vect.transform(['hi how are you? shall we meet up soon?']))

array([0])

In [ ]:
clf.predict_proba(count_vect.transform(['hi how are you? shall we meet up soon?']))